In [30]:
pip install xgboost

  Obtaining dependency information for xgboost from https://files.pythonhosted.org/packages/13/94/f73d4efcc9a0272ea9f93c03f4744a2b709172309cd0bfde1e9012776330/xgboost-2.0.1-py3-none-macosx_12_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 2.9 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [31]:
import pandas as pd
import numpy as np
import sklearn
import xgboost

In [8]:
df = pd.read_csv("DATA/final_df.csv", index_col=0)

In [9]:
df.head()

,Company,Device,Market,SubMarket,Year,Value,Spendings,DevSpends,MarketSpends,SubMarketSpends,Cogs,Cogs_1,Cogs_2,Revenue,Revenue_1,Revenue_2,Ebitda,Ebitda_1,Ebitda_2
0,AMETEK,Amplifier/Comparator,Industrial,Manufacturing Equipment,2018,1.349324,63.254,33009.633,202444.977,52627.040,3186.31,2861.37,2547.60,4845.9,4300.2,3840.1,1261.6,1068.2,1007.2
1,AMETEK,Amplifier/Comparator,Industrial,Manufacturing Equipment,2019,1.491100,58.716,31492.062,190906.584,48480.544,3370.90,3186.31,2861.37,5158.6,4845.9,4300.2,1411.4,1261.6,1068.2
2,AMETEK,Amplifier/Comparator,Industrial,Manufacturing Equipment,2020,1.362700,61.471,30177.791,197047.823,54473.792,2996.52,3370.90,3186.31,4540.0,5158.6,4845.9,1283.2,1411.4,1261.6
3,AMETEK,Amplifier/Comparator,Industrial,Manufacturing Equipment,2021,2.164928,84.752,39871.494,245502.837,72134.752,3633.90,2996.52,3370.90,5546.5,4540.0,5158.6,1600.8,1283.2,1411.4
4,AMETEK,Amplifier/Comparator,Industrial,Manufacturing Equipment,2022,2.637800,88.482,44223.186,272578.716,80606.752,4005.26,3633.90,2996.52,6150.5,5546.5,4540.0,1820.1,1600.8,1283.2


In [10]:
df.rename({"Value":"CompanySpend", "Spendings":"TotalSpend"}, axis=1, inplace=True)

In [11]:
df= pd.get_dummies(df, columns=["Device", "Market", "SubMarket"])

In [38]:
df.head()

,Company,Year,CompanySpend,TotalSpend,DevSpends,MarketSpends,SubMarketSpends,Cogs,Cogs_1,Cogs_2,...,SubMarket_Other Storage,SubMarket_Other Wired,SubMarket_Other Wireless,SubMarket_Power & Energy,SubMarket_STB,SubMarket_Smart Cards,SubMarket_TV,SubMarket_Tablet PCs,SubMarket_Test & Measurement,SubMarket_Video Games
0,AMETEK,2018,1.349324,63.254,33009.633,202444.977,52627.040,3186.31,2861.37,2547.60,...,0,0,0,0,0,0,0,0,0,0
1,AMETEK,2019,1.491100,58.716,31492.062,190906.584,48480.544,3370.90,3186.31,2861.37,...,0,0,0,0,0,0,0,0,0,0
2,AMETEK,2020,1.362700,61.471,30177.791,197047.823,54473.792,2996.52,3370.90,3186.31,...,0,0,0,0,0,0,0,0,0,0
3,AMETEK,2021,2.164928,84.752,39871.494,245502.837,72134.752,3633.90,2996.52,3370.90,...,0,0,0,0,0,0,0,0,0,0
4,AMETEK,2022,2.637800,88.482,44223.186,272578.716,80606.752,4005.26,3633.90,2996.52,...,0,0,0,0,0,0,0,0,0,0


In [27]:
X_train = df[(df.Year<=2022)]
X_test = df[(df.Year>=2023)]
y_train = X_train.pop("CompanySpend")
y_test = X_test.pop("CompanySpend")
X_train.drop(["Company", "Year"], axis=1, inplace=True)
X_test.drop(["Company", "Year"], axis=1, inplace=True)

/var/folders/px/tjlhxsg56bg6d74d3c4cnhrc0000gn/T/ipykernel_11247/3952824684.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(["Company", "Year"], axis=1, inplace=True)
/var/folders/px/tjlhxsg56bg6d74d3c4cnhrc0000gn/T/ipykernel_11247/3952824684.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(["Company", "Year"], axis=1, inplace=True)


In [42]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [43]:
model_xgb = xgb.XGBRegressor(verbosity=2, device="cuda")

In [56]:

parameters = {'learning_rate': np.arange(0.001, 0.1, 0.005), 
              'max_depth': [4, 5, 6, 7, 8],
              'min_child_weight': [1, 2, 3, 4, 5],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [100, 200, 300, 500],
              'alpha': np.arange(0, 2, 0.1),
              'lambda' : np.arange(0,2,0.1)}

xgb_grid = RandomizedSearchCV(model_xgb,
                        parameters,
                        cv = 4,
                        n_jobs = 1,
                        verbose=3,
                        n_iter = 500)


In [58]:
xgb_grid.fit(X_train,
         y_train)

Fitting 4 folds for each of 500 candidates, totalling 2000 fits
[CV 1/4] END alpha=1.9000000000000001, colsample_bytree=0.7, lambda=0.9, learning_rate=0.046, max_depth=6, min_child_weight=1, n_estimators=300, subsample=0.7;, score=0.795 total time=   0.3s
[CV 2/4] END alpha=1.9000000000000001, colsample_bytree=0.7, lambda=0.9, learning_rate=0.046, max_depth=6, min_child_weight=1, n_estimators=300, subsample=0.7;, score=-1.905 total time=   0.3s
[CV 3/4] END alpha=1.9000000000000001, colsample_bytree=0.7, lambda=0.9, learning_rate=0.046, max_depth=6, min_child_weight=1, n_estimators=300, subsample=0.7;, score=-12.552 total time=   0.4s
[CV 4/4] END alpha=1.9000000000000001, colsample_bytree=0.7, lambda=0.9, learning_rate=0.046, max_depth=6, min_child_weight=1, n_estimators=300, subsample=0.7;, score=0.384 total time=   0.3s
[CV 1/4] END alpha=0.0, colsample_bytree=0.7, lambda=1.6, learning_rate=0.021, max_depth=7, min_child_weight=5, n_estimators=100, subsample=0.7;, score=0.464 total t

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device='cuda',
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rat...
                                        'lambda': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9]),
                                        'learning_rate': array([0.001, 0.006, 0.011, 0.016, 0.021, 0.026, 0.031, 0.036, 0.041,
       0.046, 0.051, 0.056, 0.061, 0.066, 0.071, 0.076, 0.081, 0.086,
       0.091, 0.096]),
                                        'max_depth': [4, 5, 6, 7, 8],
                                        'min_child_weight': [1, 2, 3, 4, 5],
                                        'n_estimators': [100, 200, 300, 500],
                                        'subsample': [0.7]},
                   verbose=3)

In [59]:
best_params = xgb_grid.best_params_
best_score = xgb_grid.best_score_

In [60]:
best_params

{'subsample': 0.7,
 'n_estimators': 200,
 'min_child_weight': 2,
 'max_depth': 4,
 'learning_rate': 0.001,
 'lambda': 0.5,
 'colsample_bytree': 0.7,
 'alpha': 0.30000000000000004}

In [61]:
best_score

-0.03734267958655807